### Import Libraries

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() 

from langchain_community.document_loaders import CSVLoader
# embedding 
from langchain.embeddings import OllamaEmbeddings
# vector db 
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter 
from langchain_google_genai import ChatGoogleGenerativeAI

## Load Data

In [2]:
GOOGLE_API_KEY =os.getenv('Gemini_key')

In [3]:
file_path =r"..\Data\data.csv"
loader =CSVLoader(file_path)
fit_docs =loader.load()
print(fit_docs[0].page_content)

id: 0
exercise_name: Push-Ups
type_of_activity: Strength
type_of_equipment: Bodyweight
body_part: Upper Body
type: Push
muscle_groups_activated: Pectorals, Triceps, Deltoids
instructions: Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.


In [4]:
embeddings =OllamaEmbeddings(model ='plutonioumguy/bge-m3')
# vectore_store =Chroma.from_documents(
#     embedding=embeddings,
#     persist_directory='./chromadb',
#     documents =fit_docs
# )

# # 3. Save the vector store
# vectore_store.persist()

C:\Users\elkhwaga\AppData\Local\Temp\ipykernel_14716\4260143146.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings =OllamaEmbeddings(model ='plutonioumguy/bge-m3')


In [5]:
# loading chromadb from disk later
vectore_store = Chroma(
    embedding_function=embeddings,
    persist_directory='./chromadb'
)

C:\Users\elkhwaga\AppData\Local\Temp\ipykernel_14716\2351679744.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectore_store = Chroma(


In [50]:
retriver =vectore_store.as_retriever(search_kwargs={'k':5})

In [ ]:
# rag from documents
query = "Which muscles do push-ups work?"
response = retriver.get_relevant_documents(query)
response

### RAG Flow

In [80]:
llm =ChatGoogleGenerativeAI(
    model ='models/gemini-2.0-flash-lite',
    temperature=0.4,
    google_api_key=GOOGLE_API_KEY 
)

prompt_template ='\n'.join([
    "you are a fitness instructor",
    "answer the question based on the given context about exercises",
    "use only the facts from the context,when answering the question",
    "reply with a structured way"
    "Question:{question}",
    "context:{context}"
])

prompt =ChatPromptTemplate.from_template(prompt_template)

# rag chain 
fitness_chain= (
    {"question":itemgetter('question'),
     "context":lambda x:retriver.invoke(x['question'])}
    |prompt
    |llm
)


In [81]:
response = fitness_chain.invoke({"question": "Which exercise targets the core muscles?"})
print(response.content)  

The following exercises target the core muscles:

*   Mountain Climbers
*   Burpees
*   Reverse Crunch
*   Renegade Rows
*   Toe Touches


### RAG Evaluation 
__LLM As a Judge__

In [8]:
import pandas as pd
df_question =pd.read_csv('../Data/ground-truth-retrieval.csv')
df_question.head()

,id,question
0,0,What is the starting position for doing push-ups?
1,0,Which muscle groups are activated during push-...
2,0,How do you know when to push back up while doi...
3,0,Do you need any equipment to perform push-ups?
4,0,What part of the body do push-ups primarily ta...


In [9]:
ground_truth =df_question.to_dict(orient='records')
ground_truth[0]

{'id': 0, 'question': 'What is the starting position for doing push-ups?'}

In [60]:
llm_Judge=ChatGoogleGenerativeAI(
    model ='models/gemini-1.5-flash',
    temperature=0.2,
    google_api_key=GOOGLE_API_KEY 
)

Judge_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [61]:
Judge_prompt =ChatPromptTemplate.from_template(Judge_template)

Judge_chain=(
    Judge_prompt
    |llm_Judge
)

In [62]:
first_recored =ground_truth[0]
answer_llm =fitness_chain.invoke({'question':first_recored['question']})
# print(answer_llm)
response =Judge_chain.invoke({
                'question':first_recored['question'],
                'answer_llm':answer_llm.content
                })

print(response.content)

{
  "Relevance": "RELEVANT",
  "Explanation": "The answer directly and accurately addresses the question by clearly stating the starting position for push-ups.  It's concise and provides the necessary information."
}


In [63]:
type(response.content)

str

In [64]:
df_sample = df_question.sample(n=50, random_state=1)

In [65]:
sample = df_sample.to_dict(orient='records')

In [ ]:
from tqdm.auto import tqdm
import json
import time 

evaluations = []
for record in tqdm(sample):
    question = record['question']
    answer_llm = answer_llm =fitness_chain.invoke({'question':question}).content

    response =Judge_chain.invoke({
                'question':question,
                'answer_llm':answer_llm
                }).content
    print(response)
    time.sleep(2)
    evaluation = json.loads(response)
    evaluations.append((record, answer_llm, evaluation))

In [70]:
evaluations[0]

({'id': 171,
  'question': 'What is the primary muscle group activated during the Banded Pull-Up?'},
 'The primary muscle groups activated during the Banded Pull-Up are:\n\n*   Latissimus Dorsi\n*   Biceps',
 {'Relevance': 'PARTLY_RELEVANT',
  'Explanation': "The answer correctly identifies the Latissimus Dorsi and Biceps as activated muscles during banded pull-ups. However, it is incomplete as it doesn't mention other significant muscle groups involved, such as the rhomboids, trapezius, and posterior deltoids.  A more comprehensive answer would include these, making the current response only partially relevant."})

In [71]:
df_eval_50 =pd.DataFrame(evaluations,columns=['record','answer','evaluation'])
df_eval_50.head()

,record,answer,evaluation
0,"{'id': 171, 'question': 'What is the primary m...",The primary muscle groups activated during the...,"{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."
1,"{'id': 115, 'question': 'Can jumping squats be...","Yes, jumping squats can be performed without a...","{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."
2,"{'id': 53, 'question': 'Can you explain the pr...",Here's how to perform a Dumbbell Lateral Raise...,"{'Relevance': 'NON_RELEVANT', 'Explanation': '..."
3,"{'id': 198, 'question': 'How do I ensure my ar...",To ensure your arms are fully extended during ...,"{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."
4,"{'id': 19, 'question': 'What part of the body ...",Goblet Squats primarily engage the **Lower Bod...,"{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."


In [72]:
df_eval_50['id'] = df_eval_50.record.apply(lambda d: d['id'])
df_eval_50['question'] = df_eval_50.record.apply(lambda d: d['question'])

df_eval_50['relevance'] = df_eval_50.evaluation.apply(lambda d: d['Relevance'])
df_eval_50['explanation'] = df_eval_50.evaluation.apply(lambda d: d['Explanation'])

del df_eval_50['record']
del df_eval_50['evaluation']

In [73]:
df_eval_50.head()

,answer,id,question,relevance,explanation
0,The primary muscle groups activated during the...,171,What is the primary muscle group activated dur...,PARTLY_RELEVANT,The answer correctly identifies the Latissimus...
1,"Yes, jumping squats can be performed without a...",115,Can jumping squats be performed without any eq...,PARTLY_RELEVANT,The answer correctly states that jumping squat...
2,Here's how to perform a Dumbbell Lateral Raise...,53,Can you explain the proper technique for execu...,NON_RELEVANT,The generated answer provides a highly simplif...
3,To ensure your arms are fully extended during ...,198,How do I ensure my arms are fully extended dur...,PARTLY_RELEVANT,The answer provides examples of exercises that...
4,Goblet Squats primarily engage the **Lower Bod...,19,What part of the body is primarily engaged in ...,PARTLY_RELEVANT,The answer correctly identifies that the lower...


In [ ]:
df_eval_50.relevance.value_counts()

relevance
PARTLY_RELEVANT    31
RELEVANT           12
NON_RELEVANT        7
Name: count, dtype: int64